# Load The Model

In [1]:
import sys
sys.path.append('..')
import subprocess

# Specify The  TEST IDs

In [ ]:
# test_ids_base = [37,40,43,46]
# test_ids_2n2n = [38,41,44,47]
# test_ids_ours = [39,42,45,48]
test_ids_ours = [24]

path = '../results/'
# test_ids_base = [f"{path}TEMP_ID{id:03d}" for id in test_ids_base]
# test_ids_2n2n = [f"{path}TEMP_ID{id:03d}" for id in test_ids_2n2n]
test_ids_ours = [f"{path}TEST_FINAL_V2_ID{id:03d}" for id in test_ids_ours]

# Find The Best Train And Test Accuracy (top1 | top5)

In [20]:
import os

def process_test_ids(test_ids):
    results = {}

    for test_id in test_ids:
        results[test_id] = {
            "dataset": None,
            "train": {"top1": {"accuracy": None, "epoch": None}, "top5": {"accuracy": None, "epoch": None}},
            "test": {"top1": {"accuracy": None, "epoch": None}, "top5": {"accuracy": None, "epoch": None}}
        }

        model_stats_path = os.path.join(test_id, "model_stats")
        accuracy_stats_path = os.path.join(test_id, "accuracy_stats")

        model_info_file = os.path.join(model_stats_path, "model_info.txt")
        with open(model_info_file, "r") as f:
            for line in f:
                if "dataset is" in line:
                    results[test_id]["dataset"] = line.split("dataset is ")[1].strip()
                    break

        train_file = os.path.join(accuracy_stats_path, "report_train.txt")
        test_file = os.path.join(accuracy_stats_path, "report_val.txt")

        def find_best_accuracies(file_path):
            best_top1 = {"accuracy": 0, "epoch": 0}
            best_top5 = {"accuracy": 0, "epoch": 0}

            with open(file_path, "r") as f:
                for line in f:
                    if "epoch" in line:
                        epoch = int(line.split("epoch ")[1].split(":")[0])
                        top1 = float(line.split("top1%=")[1].split(",")[0])
                        top5 = float(line.split("top5%=")[1].split(",")[0])

                        if top1 > best_top1["accuracy"]:
                            best_top1["accuracy"] = top1
                            best_top1["epoch"] = epoch
                        if top5 > best_top5["accuracy"]:
                            best_top5["accuracy"] = top5
                            best_top5["epoch"] = epoch

            return best_top1, best_top5

        best_train_top1, best_train_top5 = find_best_accuracies(train_file)
        results[test_id]["train"]["top1"] = best_train_top1
        results[test_id]["train"]["top5"] = best_train_top5

        best_test_top1, best_test_top5 = find_best_accuracies(test_file)
        results[test_id]["test"]["top1"] = best_test_top1
        results[test_id]["test"]["top5"] = best_test_top5

    return results


results_base = process_test_ids(test_ids_base)
results_2n2n = process_test_ids(test_ids_2n2n)
results_ours = process_test_ids(test_ids_ours)

print(results_base)
print(results_2n2n)
print(results_ours)

{'../results/TEMP_ID037': {'dataset': 'cifar10', 'train': {'top1': {'accuracy': 0.42272499203681946, 'epoch': 199}, 'top5': {'accuracy': 0.9073999524116516, 'epoch': 183}}, 'test': {'top1': {'accuracy': 0.42544999718666077, 'epoch': 195}, 'top5': {'accuracy': 0.908549964427948, 'epoch': 180}}}, '../results/TEMP_ID040': {'dataset': 'cifar100', 'train': {'top1': {'accuracy': 0.19522500038146973, 'epoch': 193}, 'top5': {'accuracy': 0.4743749797344208, 'epoch': 194}}, 'test': {'top1': {'accuracy': 0.1934499889612198, 'epoch': 200}, 'top5': {'accuracy': 0.4632999897003174, 'epoch': 200}}}, '../results/TEMP_ID043': {'dataset': 'mnist', 'train': {'top1': {'accuracy': 0.8661249876022339, 'epoch': 194}, 'top5': {'accuracy': 0.997249960899353, 'epoch': 184}}, 'test': {'top1': {'accuracy': 0.8796666860580444, 'epoch': 180}, 'top5': {'accuracy': 0.9973666667938232, 'epoch': 155}}}, '../results/TEMP_ID046': {'dataset': 'tinyimagenet', 'train': {'top1': {'accuracy': 0.10178999602794647, 'epoch': 191

# Print the Ws

In [21]:
command = "python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID039    --dataset cifar10   --batch_size 32 --n_epoch 200 --image_size 32 --train_size 40000  --patch_size 4  --num_classes 10    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True"


try:
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    
    print("Command output (stdout):")
    print(result.stdout)
    
    if result.stderr:
        print("Command errors (stderr):")
        print(result.stderr)
    
    print(f"Command '{command}' executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Command '{command}' failed with error: {e}")
    if e.stdout:
        print("Command output (stdout):")
        print(e.stdout.decode())
    if e.stderr:
        print("Command errors (stderr):")
        print(e.stderr.decode())
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Command output (stdout):
Device is set to : cuda
No seed is set!
<All keys matched successfully>
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([1., 0., 0., 0.], device='cuda:0')
tensor([1., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([1., 0., 0., 0.], device='cuda:0')

Command 'python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID039    --dataset cifar10   --batch_size 32 --n_epoch 200 --image_size 32 --train_size 40000  --patch_size 4  --num_classes 10    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True' executed successfully.


In [22]:
command = "python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID042    --dataset cifar100   --batch_size 32 --n_epoch 200 --image_size 32 --train_size 40000  --patch_size 4  --num_classes 100    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True"

try:
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    
    print("Command output (stdout):")
    print(result.stdout)
    
    if result.stderr:
        print("Command errors (stderr):")
        print(result.stderr)
    
    print(f"Command '{command}' executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Command '{command}' failed with error: {e}")
    if e.stdout:
        print("Command output (stdout):")
        print(e.stdout.decode())
    if e.stderr:
        print("Command errors (stderr):")
        print(e.stderr.decode())
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Command output (stdout):
Device is set to : cuda
No seed is set!
<All keys matched successfully>
tensor([0., 1., 0., 0.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([1., 0., 0., 0.], device='cuda:0')

Command 'python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID042    --dataset cifar100   --batch_size 32 --n_epoch 200 --image_size 32 --train_size 40000  --patch_size 4  --num_classes 100    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True' executed successfully.


In [23]:
command = "python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID045    --dataset mnist   --batch_size 32 --n_epoch 200 --image_size 32 --train_size 40000  --patch_size 4  --num_classes 10    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True"

try:
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    
    print("Command output (stdout):")
    print(result.stdout)
    
    if result.stderr:
        print("Command errors (stderr):")
        print(result.stderr)
    
    print(f"Command '{command}' executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Command '{command}' failed with error: {e}")
    if e.stdout:
        print("Command output (stdout):")
        print(e.stdout.decode())
    if e.stderr:
        print("Command errors (stderr):")
        print(e.stderr.decode())
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Command output (stdout):
Device is set to : cuda
No seed is set!
<All keys matched successfully>
tensor([0., 1., 0., 0.], device='cuda:0')
tensor([1., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([0., 1., 0., 0.], device='cuda:0')

Command 'python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID045    --dataset mnist   --batch_size 32 --n_epoch 200 --image_size 32 --train_size 40000  --patch_size 4  --num_classes 10    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True' executed successfully.


In [24]:
command = "python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID048    --dataset tinyimagenet   --batch_size 64 --n_epoch 200 --image_size 32 --train_size 70000  --patch_size 4  --num_classes 200    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True"

try:
    result = subprocess.run(command, shell=True, check=True, capture_output=True, text=True)
    
    print("Command output (stdout):")
    print(result.stdout)
    
    if result.stderr:
        print("Command errors (stderr):")
        print(result.stderr)
    
    print(f"Command '{command}' executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Command '{command}' failed with error: {e}")
    if e.stdout:
        print("Command output (stdout):")
        print(e.stdout.decode())
    if e.stderr:
        print("Command errors (stderr):")
        print(e.stderr.decode())
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Command output (stdout):
Device is set to : cuda
No seed is set!
<All keys matched successfully>
tensor([0., 0., 1., 0.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')
tensor([0., 1., 0., 0.], device='cuda:0')
tensor([0., 0., 0., 1.], device='cuda:0')

Command 'python  ../TESTS/TEST_glvit_v7_val.py    --TEST_ID TEMP_ID048    --dataset tinyimagenet   --batch_size 64 --n_epoch 200 --image_size 32 --train_size 70000  --patch_size 4  --num_classes 200    --dim 64    --depth 6   --heads 8   --mlp_dim 128   --debug True' executed successfully.
